In [1]:
#import statements
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime as dt
import math
import json

here = os.getcwd()
flora_h = F"{here}/resources/reference-data/"
flora_h_ws = F"{flora_h}atlasws/"
flora_h_55 = F"{flora_h}atlas5x5/"
data_2020 = F"{here}/resources/survey-data/"

## Harmonize taxonomy and aggregate results from different data sources

The data for the observations is collected by different groups and aggregated at different geographic scales. Here all the data is categorized by *genus-species*, the subspcieces are accounted for and the survey results froom 2020 are harmonized with the prior observations.

In [2]:
my_data_methods = {"csv":pd.read_csv}

# organize the data sources into dicts one for each directory
d_files = {
    "surveys":"surveys.csv",
    "map_keys":"map-keys.csv",    
}

w_lists = {
    "list_2014":"BL_WL_2014_modified.csv",
    "under_sampled":"taxa_sous_echantillonnes.csv",
    "red_list":"redlist2019.csv",
    "cert_list":"Certification_specieslist_2021.csv"
}

ws_lists ={
    "151":"AtlasWS_151_Biel.csv",
    "253":"AtlasWS_253_Bielersee.csv",
    "300":"AtlasWS_300_Aarberg.csv",
    "301":"AtlasWS_301_Bueren.csv",
    "154":"AtlasWS_154_Grenchen.csv",
    "572":"AtlasWS_572_Beatenberg.csv",
    "573":"AtlasWS_573_Interlaken.csv"   
}

fx_lists = {
    "585215":"Atlas5x5_585_215.csv",
    "585220":"Atlas5x5_585_220.csv",
    "580220":"Atlas5x5_580_220.csv",
    "580215":"Atlas5x5_580_215.csv"
}

# convenience method to gather up all these files:
def get_the_data(file_exts, a_dir, methods, this_method="csv", myencoding=None):
    wiw = {}
    for k,v in file_exts.items():
        if myencoding == None:
            wiw.update({k:methods[this_method](F"{a_dir}{v}")})            
        else:
            wiw.update({k:methods[this_method](F"{a_dir}{v}",sep = ";", encoding=myencoding)})
            
    return wiw

# convenience method to make slugs
def change_species(x):
    # make a slug
    try: 
        int_data = x.split()
        data = int_data[:2]
        data = "-".join(data)
        data = data.lower()
    except:
        data = "none"
    return data


data_and_keys = get_the_data(d_files, data_2020, my_data_methods, this_method="csv")
watch_lists = get_the_data(w_lists, flora_h, my_data_methods, this_method="csv")
welt_sut =  get_the_data(ws_lists, flora_h_ws, my_data_methods, this_method="csv", myencoding = "utf-16" )
fivex =  get_the_data(fx_lists, flora_h_55, my_data_methods, this_method="csv", myencoding = "utf-16")

# housekeeping: make sure that each data set has the column "species", with the value species:
watch_lists["list_2014"]["species"] = watch_lists["list_2014"].Latin
watch_lists["under_sampled"]["species"] = watch_lists["under_sampled"].taxon
watch_lists["red_list"]["species"] = watch_lists["red_list"].scientific_name
watch_lists["cert_list"]["species"] = watch_lists["cert_list"]["Short Name"]

# housekeeping: fx_lists['585220'] has one record in one column of one row stored as a string
adf = fivex['585220'].copy()

# # housekeeping: watch_lists has a NaN value for the name of a watchlist
# x= watch_lists["list_2014"]['watch_lists'].fillna(0)


# get the string value, split it by the comma and make a dict
def make_this_a_dict(x, these_cols):
    return {these_cols[i]:j for i, j in enumerate(x.split(','))}

# make a column with that
adf['a_dict']=adf[adf.columns[0]].map(lambda x:make_this_a_dict(x, fivex['585215'].columns))

# make a dataframe with that column, replace the old data frame:
fivex['585220'] = pd.DataFrame(list(adf['a_dict'].values))

In [38]:
watch_lists["list_2014"]

,Latin,French,Jura,Plateau,Versant Nord des Alpes,Alpes centrales Ouest,Alpes centrales Est,Versant Sud des Alpes,non établi en Suisse,1 Eau1 libres,...,"écologie, biodiversité",économie,"Präventionscharakter (hoch=1, mittel=2, gering=3)","Regionale Wichtigkeit (hoch=1, mittel=2, gering=3)","Lack List / Watch List ""old""",list_2014,Ordonnonce sur la dissémination des organismes (ODE),species,species_slug,watch_list
0,Abutilon theophrasti Medik.,Abutilon de Théophraste,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1,3.0,2,2,-,BL,-,Abutilon theophrasti Medik.,abutilon-theophrasti,BL
1,Ailanthus altissima (Mill.) Swingle,Ailante,2.0,3.0,1.0,2.0,1.0,3.0,0.0,0.0,...,2,2.0,3,3,BL,BL,-,Ailanthus altissima (Mill.) Swingle,ailanthus-altissima,BL
2,Ambrosia artemisiifolia L.,Ambroisie à feuilles d'armoise,3.0,3.0,1.0,2.0,1.0,3.0,0.0,0.0,...,-1,3.0,3,2,BL,BL,ODE,Ambrosia artemisiifolia L.,ambrosia-artemisiifolia,BL
3,Amorpha fruticosa L.,Amorphe buissonnante,1.0,-1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3,2.0,1,1,WL,BL,-,Amorpha fruticosa L.,amorpha-fruticosa,BL
4,Artemisia verlotiorum Lamotte,Armoise des frères Verlot,2.0,3.0,2.0,2.0,1.0,3.0,0.0,0.0,...,2,2.0,3,3,BL,BL,-,Artemisia verlotiorum Lamotte,artemisia-verlotiorum,BL
5,Asclepias syriaca L.,Asclépiade de Syrie,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3,3.0,1,1,WL,BL,-,Asclepias syriaca L.,asclepias-syriaca,BL
6,Buddleja davidii Franch.,Buddléia de David,3.0,3.0,3.0,2.0,2.0,3.0,0.0,0.0,...,3,1.0,3,3,BL,BL,-,Buddleja davidii Franch.,buddleja-davidii,BL
7,Bunias orientalis L.,Bunias d'Orient,3.0,2.0,0.0,3.0,2.0,1.0,0.0,0.0,...,2,3.0,2,2,WL,BL,-,Bunias orientalis L.,bunias-orientalis,BL
8,Cabomba caroliniana A. Gray,"Cabomba, Evantail de Caroline",0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,3,3.0,1,0,-,BL,-,Cabomba caroliniana A. Gray,cabomba-caroliniana,BL
9,Crassula helmsii (Kirk) Cockayne,Crassule de Helm,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,3,3.0,1,0,-,BL,ODE,Crassula helmsii (Kirk) Cockayne,crassula-helmsii,BL


In [37]:
# create a new column to hold a species 'slug'
for element in [fivex, welt_sut, watch_lists]:
    for the_data in element:
        element[the_data]['species_slug'] = 'none'

# make the species slug
for element in [fivex, welt_sut, watch_lists]:
    for the_data in element:
#         print(element.keys())
#         print(the_data)
#         if the_data == "cert_list":
#             pass
#         else:
        element[the_data]['species_slug'] = element[the_data].species.map(lambda x: change_species(x))

# add a column to identify the map source:
for element in [fivex, welt_sut]:
    for the_data in element:
        element[the_data]['map'] = the_data
        element[the_data]['spec_map'] = list(zip(element[the_data].species_slug,element[the_data].map))
        
# add a column to identify watch list:
for element in [watch_lists]:
    for the_data in element:
        print(element.keys())
        print(the_data)
        if the_data == "list_2014":
            element[the_data]['watch_list'] = element[the_data][the_data]
        else:
            element[the_data]['watch_list'] = the_data
        

dict_keys(['list_2014', 'under_sampled', 'red_list', 'cert_list'])
list_2014
dict_keys(['list_2014', 'under_sampled', 'red_list', 'cert_list'])
under_sampled
dict_keys(['list_2014', 'under_sampled', 'red_list', 'cert_list'])
red_list
dict_keys(['list_2014', 'under_sampled', 'red_list', 'cert_list'])
cert_list


/home/mwshovel/anaconda3/envs/iqals/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/mwshovel/anaconda3/envs/iqals/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/mwshovel/anaconda3/envs/iqals/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [6]:
fill_nans = watch_lists["list_2014"].copy()
fill_nans = fill_nans.fillna(0)

watch_lists.update({"list_2014":fill_nans[fill_nans.watch_list != 0]})

### Species name and observations: harmonizing taxonomy

The genus-species nomenclature will be used to group observations.

All observations will be classified according to that standard. As a result some subspecies will be folded in with the parent genus-species. This is a reflection of the survey method and the expectation of reasonable results, not a prioritization of importance. The complete list of genus species and subspecies is included at the end of this document.

In [7]:
def account_for_subspecies(an_array, a_dict):
    for element in an_array:
        try:
            a_dict[element[0]].append(element[1])
        except:
            a_dict[element[0]] = [element[1]]
    return a_dict
a_dict ={}

for element in [fivex, welt_sut, watch_lists]:
    for label in element:
        # use this data frame
        som_data = element[label].copy()
        
        # group by species slug and count the number of unique species values
        c_s_p_s = som_data.groupby('species_slug', as_index=False).species.nunique()        
        
        # just the records with more than one species value
        m_t_one = c_s_p_s[c_s_p_s.species > 1].species_slug
        
        # pair the species_slug to the species name:
        mto = som_data.loc[som_data.species_slug.isin(m_t_one)][['species_slug', 'species']].copy().to_numpy()
        
        # update the dict
        account_for_subspecies(mto, a_dict)

# the species_slugs that account for more than one sub species
sub_species_accounted =  {k:list(set(v)) for k,v in a_dict.items()}

# the species_slugs
gs_parent = list(sub_species_accounted.keys())

# the number of species_slugs
number_of_gs = len(gs_parent)

# the the number of sub species accounted for
number_of_ss = sum([len(v) for k,v in sub_species_accounted.items()])

In [8]:
print(F"\nThere are {number_of_gs} genus-species that account for {number_of_ss} sub-species. For example records with 'trifolium-incarnatum' include:\n\n{sub_species_accounted['trifolium-incarnatum']}\n")
print("In this specific example all records with the genus-species of 'trifolium-incarnatum' will be counted as the same, indifferent of the sub-species value.")


There are 323 genus-species that account for 1156 sub-species. For example records with 'trifolium-incarnatum' include:

['Trifolium incarnatum L.', 'Trifolium incarnatum L. subsp. incarnatum']

In this specific example all records with the genus-species of 'trifolium-incarnatum' will be counted as the same, indifferent of the sub-species value.


### Collect genus-species observations from each map set.

In [9]:
# collect all the observations into one df
fx = pd.concat([v[['species_slug', 'map', 'spec_map']] for k,v in fivex.items()])
wsx = pd.concat([v[['species_slug', 'map', 'spec_map']] for k,v in welt_sut.items()])
f_w_obs = pd.concat([fx, wsx])

In [10]:
# gather up the map names
m_ap_columns = f_w_obs.map.unique()

# create a column for each map, indicate
for col in m_ap_columns:    
    f_w_obs[col] = f_w_obs['map'] == col

obs_map =f_w_obs.groupby(['species_slug']).sum()

obs_map['maps'] = obs_map.index.map(lambda x: f_w_obs[f_w_obs.species_slug == x]['map'].unique())

#### The genus-species and the maps it was identified on:

In [11]:
obs_map

,585215,585220,580220,580215,151,253,300,301,154,572,573,maps
species_slug,,,,,,,,,,,,
abies-alba,1,1,1,1,1,0,1,1,1,1,1,"[585215, 585220, 580220, 580215, 151, 300, 301..."
abies-nordmanniana,0,0,0,0,0,0,0,0,1,0,0,[154]
abutilon-theophrasti,1,1,0,0,1,0,1,1,0,0,0,"[585215, 585220, 151, 300, 301]"
acanthus-mollis,0,0,0,0,1,0,0,0,0,0,0,[151]
acer-campestre,1,1,1,1,1,1,1,1,1,1,1,"[585215, 585220, 580220, 580215, 151, 253, 300..."
...,...,...,...,...,...,...,...,...,...,...,...,...
xanthium-spinosum,0,0,1,0,1,0,0,0,0,0,0,"[580220, 151]"
xanthium-strumarium,2,2,2,0,2,0,2,0,2,0,0,"[585215, 585220, 580220, 151, 300, 154]"
yucca-filamentosa,0,0,0,0,0,0,0,0,1,0,0,[154]


In [12]:
print(F"\nFor example 'abies-alba' was identified in the following maps:\n\n{obs_map.loc['abies-alba']['maps']}\n")


For example 'abies-alba' was identified in the following maps:

['585215' '585220' '580220' '580215' '151' '300' '301' '154' '572' '573']



In [13]:
print(F"\nBack to the trifolium example:\n")
print("Not all subspecies were idenitfied on all the maps\n")
obs_map.loc['trifolium-incarnatum']


Back to the trifolium example:

Not all subspecies were idenitfied on all the maps



585215                                          2
585220                                          2
580220                                          0
580215                                          1
151                                             2
253                                             0
300                                             2
301                                             0
154                                             2
572                                             0
573                                             0
maps      [585215, 585220, 580215, 151, 300, 154]
Name: trifolium-incarnatum, dtype: object

In [14]:
#exports the dictionary to a .json file
nt = obs_map['maps'].to_dict()

ntx = {k:list(v) for k,v in nt.items()}

with open(F"{here}/output/ws_list.json","w") as afile:
    json.dump(ntx,afile)
    
print(F"\nWhich maps 'trifolium-incarnatum' were found in? indifferent of subspecies?:\n\n{ntx['trifolium-incarnatum']}\n")


Which maps 'trifolium-incarnatum' were found in? indifferent of subspecies?:

['585215', '585220', '580215', '151', '300', '154']



### Key the watch lists to genus-species

Identify which species are on which watch list. Produce a boolean matrix with species_slug as index and watch list name for the columns.

In [35]:
watch_lists.keys()
watch_lists["list_2014"]

,Latin,French,Jura,Plateau,Versant Nord des Alpes,Alpes centrales Ouest,Alpes centrales Est,Versant Sud des Alpes,non établi en Suisse,1 Eau1 libres,...,"écologie, biodiversité",économie,"Präventionscharakter (hoch=1, mittel=2, gering=3)","Regionale Wichtigkeit (hoch=1, mittel=2, gering=3)","Lack List / Watch List ""old""",list_2014,Ordonnonce sur la dissémination des organismes (ODE),species,species_slug,watch_list
0,Abutilon theophrasti Medik.,Abutilon de Théophraste,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1,3.0,2,2,-,BL,-,Abutilon theophrasti Medik.,abutilon-theophrasti,BL
1,Ailanthus altissima (Mill.) Swingle,Ailante,2.0,3.0,1.0,2.0,1.0,3.0,0.0,0.0,...,2,2.0,3,3,BL,BL,-,Ailanthus altissima (Mill.) Swingle,ailanthus-altissima,BL
2,Ambrosia artemisiifolia L.,Ambroisie à feuilles d'armoise,3.0,3.0,1.0,2.0,1.0,3.0,0.0,0.0,...,-1,3.0,3,2,BL,BL,ODE,Ambrosia artemisiifolia L.,ambrosia-artemisiifolia,BL
3,Amorpha fruticosa L.,Amorphe buissonnante,1.0,-1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3,2.0,1,1,WL,BL,-,Amorpha fruticosa L.,amorpha-fruticosa,BL
4,Artemisia verlotiorum Lamotte,Armoise des frères Verlot,2.0,3.0,2.0,2.0,1.0,3.0,0.0,0.0,...,2,2.0,3,3,BL,BL,-,Artemisia verlotiorum Lamotte,artemisia-verlotiorum,BL
5,Asclepias syriaca L.,Asclépiade de Syrie,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,3,3.0,1,1,WL,BL,-,Asclepias syriaca L.,asclepias-syriaca,BL
6,Buddleja davidii Franch.,Buddléia de David,3.0,3.0,3.0,2.0,2.0,3.0,0.0,0.0,...,3,1.0,3,3,BL,BL,-,Buddleja davidii Franch.,buddleja-davidii,BL
7,Bunias orientalis L.,Bunias d'Orient,3.0,2.0,0.0,3.0,2.0,1.0,0.0,0.0,...,2,3.0,2,2,WL,BL,-,Bunias orientalis L.,bunias-orientalis,BL
8,Cabomba caroliniana A. Gray,"Cabomba, Evantail de Caroline",0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,3,3.0,1,0,-,BL,-,Cabomba caroliniana A. Gray,cabomba-caroliniana,BL
9,Crassula helmsii (Kirk) Cockayne,Crassule de Helm,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,3,3.0,1,0,-,BL,ODE,Crassula helmsii (Kirk) Cockayne,crassula-helmsii,BL


In [16]:
the_lists = list(watch_lists.keys())

keep = []

for element in the_lists:
    a = watch_lists[element][['species_slug','watch_list']]
    keep.append(a)
wl_species = pd.concat(keep)
m_ap_columns = wl_species.watch_list.unique()

for col in m_ap_columns:    
    wl_species[col] = wl_species['watch_list'] == col

wl_sp_map =wl_species.groupby(['species_slug']).sum()

wl_sp_map['lists'] = wl_sp_map.index.map(lambda x: wl_species[wl_species.species_slug == x]['watch_list'].unique())

In [32]:
m_ap_columns

array(['BL', 'WL', 'under_sampled', 'red_list', 'cert_list'], dtype=object)

In [33]:
wl_species

,species_slug,watch_list,BL,WL,under_sampled,red_list,cert_list
0,abutilon-theophrasti,BL,True,False,False,False,False
1,ailanthus-altissima,BL,True,False,False,False,False
2,ambrosia-artemisiifolia,BL,True,False,False,False,False
3,amorpha-fruticosa,BL,True,False,False,False,False
4,artemisia-verlotiorum,BL,True,False,False,False,False
...,...,...,...,...,...,...,...
595,viola-calcarata,cert_list,False,False,False,False,True
596,viola-hirta,cert_list,False,False,False,False,True
597,viola-reichenbachiana,cert_list,False,False,False,False,True
598,viola-tricolor,cert_list,False,False,False,False,True


In [18]:
print("\nThe list of genus-species that appear in the black list and gbif:\n\n")

wl_sp_map[(wl_sp_map.BL >0)&(wl_sp_map.red_list >0)]


The list of genus-species that appear in the black list and gbif:




,BL,WL,under_sampled,red_list,cert_list,lists
species_slug,,,,,,


In [19]:
nt = wl_sp_map['lists'].to_dict()

ntx = {k:list(v) for k,v in nt.items()}

with open(F"{here}/output/flora_list.json","w") as afile:
    json.dump(ntx,afile)
wl_sp_map.to_csv(F"{here}/output/aggregated_list.csv",index=True)

### Taxonomy of 2020 hd samples

In [20]:
#correct wrong species name inputs in the samples data
#samples list
samples = data_and_keys['surveys'].copy()
replacedict = {
    'verbanum bonariensis ':'verbena bonariensis',
    'medicago varia':'medicago sativa',
    "oenothera":"oenothera biennis",
    "geranium pratens":"geranium pratense",
    "senecio jacobaea": "jacobaea vulgaris",
    "oenothera biennis ": "oenothera biennis",
    "oenothera biennis agg.": "oenothera biennis",
    "solidalgo canadensis": "solidago canadensis",
    "verbascum lynchitis":"verbascum lychnitis",
    "verbascum negris":"verbascum nigrum",
    "securigea varia": "securigera varia",
    "melilotus officianalis": "melilotus officinalis",
    "knautia maxima": "knautia dipsacifolia",
    "hieracium aurantiacum":"pilosella aurantiaca",
    "sysimbrium officinale":"sisymbrium officinale",
    "geranium robertanium":"geranium robertianum",
    "mycelis muralis": "lactuca muralis",
    "calamintha-nepeta":"clinopodium nepeta",
    "polygonum-persicaria":"persicaria maculosa",
    "sorbus-aria":"aria edulis",
    "taraxacum": "taraxacum officinale"
}
def new_func(x,keys):
    try:
        data = keys[x]
    except:
        data = x
    return data
samples["species2"] = samples.sci.map(lambda x: new_func(x, replacedict))
samples["species_slug"] = samples.species2.map(lambda x: change_species(x))

data_and_keys.update({'surveys':samples})

In [21]:
#correct incorrectly formatted date inputs in the sample dataset
def change_string(x):
    try:
        s_data = x.split('.')
        data = s_data[::-1]
        data = "-".join(data)
    except:
        print("no luck")
        data = x
    
    return data
samples['new_date'] = samples.date.map(lambda x: change_string(x))


def make_timestamp(x):
    try:        
        data = dt.datetime.strptime(x, "%Y-%m-%d")        
    except:        
        data = 'no luck'
    
    return data
# samples['check_date'] = samples.new_date.map(lambda x: make_timestamp(x))
samples['date'] = samples.new_date

print(F"\nThese are the records in 'a' with no valid date:\n\n{samples[samples.date == 'no luck']}\n")


These are the records in 'a' with no valid date:

Empty DataFrame
Columns: [place, date, time, sci, name, species2, species_slug, new_date]
Index: []



In [22]:
d=data_and_keys['surveys'].copy()
d['loc_date'] = list(zip(d.place, d.date))
slugs_2020 = d.species_slug.unique()


a_name =  slugs_2020[12]
one_slug = d[d.species_slug == a_name].copy()

maps_and_lists = pd.concat([obs_map.loc[a_name],wl_sp_map.loc[a_name]])
maps_and_lists

585215                                                           2
585220                                                           2
580220                                                           3
580215                                                           2
151                                                              3
253                                                              1
300                                                              2
301                                                              3
154                                                              3
572                                                              2
573                                                              3
maps             [585215, 585220, 580220, 580215, 151, 253, 300...
BL                                                               0
WL                                                               0
under_sampled                                                 

In [23]:
obs_map.loc[a_name][:-1].sum()

26

In [24]:
dname = d[d.species_slug == a_name]
dname.loc_date

15              (alleestrasse 3, 2020-06-16)
25              (alleestrasse 4, 2020-06-16)
208        (chemin de la course, 2020-08-30)
329                  (energie 1, 2020-06-16)
411                (gottstatt 3, 2020-08-24)
487                 (rondchatel, 2020-09-22)
507              (rue de leau 1, 2020-08-23)
539              (rue de leau 4, 2020-08-17)
551              (rue de leau 5, 2020-08-18)
605           (schlosslistrasse, 2020-06-18)
708     (sundgraben bridge left, 2020-09-05)
729    (sundgraben forest right, 2020-09-13)
762                    (sureaux, 2020-08-09)
Name: loc_date, dtype: object

In [25]:
len(dname)/len(d)

0.013903743315508022

In [26]:
len(f_w_obs[f_w_obs.species_slug == a_name])/len(f_w_obs)

0.002009894867037724

In [27]:
#sample place columns

#makes place slug in new column "place1" for the locations with more than one word.
def change_place(x):
    data = x.split(" ")
    data = "-".join(data)
    return data
samples["place1"] = samples.place.map(lambda x: change_place(x))


In [28]:
samples.head()

,place,date,time,sci,name,species2,species_slug,new_date,place1
0,alleestrasse 1,2020-09-02,11:09:04 AM,plantago lanceolata,plantain lancéolé,plantago lanceolata,plantago-lanceolata,2020-09-02,alleestrasse-1
1,alleestrasse 1,2020-09-02,11:08:19 AM,centaurea nigra,centaurée noire,centaurea nigra,centaurea-nigra,2020-09-02,alleestrasse-1
2,alleestrasse 1,2020-09-02,11:06:59 AM,plantago media,plantain moyen,plantago media,plantago-media,2020-09-02,alleestrasse-1
3,alleestrasse 1,2020-09-02,11:06:05 AM,chenopodium album agg.,chénopode blanc,chenopodium album agg.,chenopodium-album,2020-09-02,alleestrasse-1
4,alleestrasse 1,2020-09-02,11:05:18 AM,centaurea jacea agg.,centaurée jacée,centaurea jacea agg.,centaurea-jacea,2020-09-02,alleestrasse-1


In [29]:
re_name = {'sci': 'genus_species', 'place1':'place_slug'}
samples.rename(columns=re_name, inplace=True)
samples.drop(['species2', 'date'] , inplace=True, axis=1)
samples.rename(columns={'new_date':'date'}, inplace=True)

In [30]:
samples = samples[['place', 'date', 'genus_species', 'time', 'species_slug', 'place_slug' ]].copy()
samples.to_csv(F"output/mostrecent/hd_samples_2020.csv", index=False)